In [ ]:
'''
Load the service of Google Colab -- use Colab GPU for training

'''

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
'''
Link to Google Drive root
'''

!mkdir -p drive
!google-drive-ocamlfuse drive
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1'

In [ ]:
'''
Import required packages
'''

from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

import numpy as np
import pandas as pd

from tensorflow import keras
import keras
import keras.backend as K
from keras.layers import TimeDistributed, Dense, LSTM, RepeatVector, Dropout, Input, Flatten,Concatenate
from keras.models import load_model
from keras.models import Model
from keras import optimizers
from tensorflow import optimizers

import matplotlib.pyplot as plt
import glob,os
import random
import os,shutil

In [ ]:
def listpath(path, list_name):  
       """List name of all csv files in a folder
        Parameters
        ----------
        path : 
            Path of root folder
        Returns
        -------
        list
            The name list of csv file in the path
        """
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if os.path.isdir(file_path):
            listdir(file_path, list_name)
        else:
            list_name.append(file_path)
        return list_name

def custom_regional_nse(y_true, y_pred):
    """Cusmomized batch-wise loss function
        Parameters
        ---------
        y_true:
            Ground truth observation
        y_pred:
            Output from batch-wise prediction
        Returns
        --------
        Normalized batch-wise loss

    """
    numerator = K.square(y_pred - y_true)  # (batch_size, 2)
    dominator = K.square(y_true - K.mean(y_true)) + 0.1
    # summing both loss values along batch dimension 
    numerator_sum = K.sum(numerator, axis=1)        # (batch_size,)
    dominator_sum = K.sum(dominator, axis=1)        # (batch_size,)
    loss = (numerator_sum)/(dominator_sum)
    return loss # automatical average along batch size when optimization

In [ ]:
def split_input(static, dataset, n_input, n_output, scaler, shuffle): # 
        """Prepare structured and paired in/output sequence for LSTM model
        Parameters
        ---------
        static:
            14 catchment-wise static variables
        dataset:
            The name list of csv file in the path
        n_input:
            length of input sequence (i.e. 24 hours)
        n_output:
            length of output sequence (i.e. 6 hours)
        scaler:
            pre-defined Min-Max scaler for input data
        shuffle: Boolean
            True or False
        Returns
        --------
        x1: np.array
            runoff observation data, paired with x2, y, static_new for each prediction
        x2:
            rainfall, tmax, tmin observation data (input + output length)
        y:
            target runoff data
        static_new:
            matrix of static variables (14, input + output length)
    """
    x1, x2, y, seq_static = list(),list(), list(), list()
    for j in range(len(dataset)): # go through each catchment
        data_1 = pd.read_csv(dataset[j], index_col = 0) 
        data_2 = pd.read_csv(dataset[j], index_col = 0) # read again for non-scalered target
        
        sequence = scaler.transform(data_1) # introduce MinMax scaler for individual event, only for input data
        cat = dataset[j].split('-')[3]
        catchment = cat.split('.')[0] # extract the catchment name
        temp_static = np.zeros((1,14))
        temp_static = static[catchment].values # each basin one static input only one step

        for i in range(len(sequence)): # go through each event
            end_ix1 = i + n_input # runoff observation data
            end_ix2 = i + n_input + n_output # rainfall, tmax, tmin observation data (input + output length)
            end_y = end_ix1 + n_output
            # check if we are beyond the sequence
            if end_ix1 > len(sequence):
                break
            if end_ix2 > len(sequence):
                break
            if end_y > len(sequence):
                break
            seq_x1, seq_y = sequence[:,0][i:end_ix1], data_2.values[:,0][end_ix1:end_y] # input runoff observation and target runoff 
            seq_x2 = sequence[:,1:][i:end_ix2, :] # input rainfall, tmax and tmin observation
            x1.append(seq_x1)
            x2.append(seq_x2)
            y.append(seq_y)
            seq_static.append(list(temp_static))

    x1, x2,y,seq_static = np.array(x1), np.array(x2), np.array(y), np.array(seq_static)
    number_list = np.arange(0,len(x1),1)
    random.shuffle(number_list) 
    x1_new = []
    for index in number_list:
        x1_new.append(x1[index])
    x2_new = []
    for index in number_list:
        x2_new.append(x2[index])
    y_new = []
    for index in number_list:
        y_new.append(y[index])
    static_new = []
    for index in number_list:
        static_new.append(seq_static[index])
    #  only shuffle the sliced in/output sequence when training model
    if shuffle:
        return np.array(x1_new), np.array(x2_new), np.array(y_new), np.array(static_new)
    else:
        return x1, x2, y, static_new

In [ ]:
nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name.csv',index_col = 0)
basins = nse_table.columns # collect name of each basin

# read 14 catchment static variables (PC scores)
pca_score = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/pca_score.csv',index_col = 0)
temp2 = pca_score.values
temp_nor = preprocessing.normalize(temp2, norm='l2',axis = 1) 
pca_score_nor = pd.DataFrame(temp_nor, index=[pca_score.index], columns=pca_score.columns)

In [ ]:
"""
When training a regional model, flood events at each catchment are preliminaryly assigned portionly (50/25/25) into three folders
-- train1, val, and test for a convenient use
"""

train_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/train1/'
val_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/val/'
test_path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/test/'

trainlist=os.listdir(train_path)
vallist=os.listdir(val_path)

# Create scaler
df2 = pd.DataFrame()
for j in range(len(trainlist)):
  for basin in basins:
    if trainlist[j].find(basin) != -1:
      data = pd.read_csv(train_path + trainlist[j], index_col = 0)
      df2 = pd.concat([df2,data],axis=0) 
temp = df2.values
min_max_scaler1 = MinMaxScaler() 
min_max_scaler1.fit(temp) 

# input for training model
train1_x, train2_x, train_y, train_static = split_input(pca_score_nor,train_path, trainlist, 24, 6,min_max_scaler1,shuffle =True)
# input for validation model
val1_x, val2_x, val_y, val_static = split_input(pca_score_nor,val_path, vallist, 24, 6, min_max_scaler1,shuffle =True)

train1_x = train1_x.reshape((train1_x.shape[0],train1_x.shape[1],1))
val1_x = val1_x.reshape((val1_x.shape[0],val1_x.shape[1],1)) 

In [ ]:
"""
LSTM model structure and training; the hyperparameter selection has been skipped here (pre-trained model)
"""

######## design LSTM network structure ########
dim_dense=[512, 256, 256, 128, 64]
# define customer optimizer
RMSprop= optimizers.RMSprop(lr=0.0001)
# define special callbacks
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=15, cooldown=30, min_lr=1e-8)
# define early stop
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, verbose=2)

# input of runoff observation, LSTM encoder
input_1 = Input(shape=(train1_x.shape[1],train1_x.shape[2]), name='LSTM1_input') # shape should be 72*1 for runoff observation
LSTM1 = LSTM(256, return_sequences=False)(input_1)

# input of runoff observation and forecast, LSTM encoder
input_2 = Input(shape=(train2_x.shape[1],train2_x.shape[2]), name='LSTM2_input') # shape should be (72+24)*n=96*n, for rainfall observation (72) and predictions (24) for n stations (if there is no upstream station, n=1)
LSTM2 = LSTM(256, return_sequences=False)(input_2)

# input of other non-timeseries data, such as daily or monthly data.
input_static = Input(shape=(14,), name='static_input') # shape = 14 -- 14 static variables -- 2D in this case with only Input or Dense layer (14,)/3D for LSTM layer (1,14)
Layer3 = Dense(16, activation='relu')(input_static)

# connect all data
x = Concatenate()([Layer3, LSTM1, LSTM2]) 
x = RepeatVector(6)(x)  # 6 hour ahead prediction

# LSTM decoder
x = LSTM(512, return_sequences=True)(x)

# final fully-connected layer for final result
for dim in dim_dense:
	x = TimeDistributed(Dense(dim, activation='relu'))(x)
	x = TimeDistributed(Dropout(0.4))(x) # Some dropout for dense layers.
main_out = TimeDistributed(Dense(1, activation='relu'))(x) 
main_out = Flatten()(main_out)

################## model training ####################
model = Model(inputs=[input_static, input_1, input_2], outputs=main_out)
model.compile(optimizer=RMSprop, loss=custom_regional_nse) 
model.summary()

# fit network 
history = model.fit([train_static, train1_x, train2_x], train_y, epochs=300, batch_size=128,
					validation_data=([val_static, val1_x, val2_x], val_y), callbacks=[reduce_lr,early_stopping], verbose=1)

In [ ]:
# visualize the train/val loss
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title( 'model train vs validation loss') 
plt.ylabel('loss') 
plt.xlabel('epoch') 
plt.legend([ 'train', 'validation'], loc= 'upper right') 
plt.show()

In [ ]:
model.save('/content/drive/My Drive/master_thesis/_code/regional_model/regional.h5' ) 

In [ ]:
###### model evaluation ######
model_regional = load_model('/content/drive/My Drive/master_thesis/_code/regional_model/regional.h5' % (num_ite), 
                           custom_objects={'custom_regional_nse': custom_regional_nse})

nse_table = pd.read_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/basin_name.csv',index_col = 0)
basins = nse_table.columns

for catchment in basins:

    path = '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/model1/%s_events/' % catchment
    file_all = sorted(os.listdir(path))
    split2 = int(len(file_all)*0.75)  
    dataset = file_all[split2:] 
    ed_nse = []
    ed_hour = []
    ed_volume = []
    ed_peak = []
    dates = []

    for i in range(len(dataset)):
        # pre-processing the input sequence for performance evaluation -- event-wise
            x1, x2,y, seq_static = list(),list(), list(), list()
            data_1 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0)
            data_2 = pd.read_csv(test_path+'/'+dataset[j], index_col = 0) # not normalize y(target)
            date = dataset[j][:10]

            # MinMax for individual event
            n_input = 24
            n_output = 6
            sequence = min_max_scaler1.transform(data_1) # same scaler as training dataset

            # add static variable
            temp_static = np.zeros((1,14))
            temp_static = pca_score_nor[catchment].values # each basin one static input only one step
            for i in range(len(sequence)):
                # find the end of this pattern
                end_ix1 = i*n_output + n_input # runoff
                end_ix2 = i*n_output + n_input + n_output # others
                end_y = end_ix1 + n_output
                # check if we are beyond the sequence
                if end_ix1 > len(sequence):
                    break
                if end_ix2 > len(sequence):
                    break
                if end_y > len(sequence):
                    break
                seq_x2 = sequence[:,1:][i*n_output:end_ix2, :] 
                seq_x1, seq_y = sequence[:,0][i*n_output:end_ix1], data_2.values[:,0][end_ix1:end_y] 
                x1.append(seq_x1)
                x2.append(seq_x2)
                y.append(seq_y)
                seq_static.append(list(temp_static))

            #event-wise model performance evaluation
            x1, x2, y, seq_static = np.array(x1), np.array(x2), np.array(y), np.array(seq_static)
            predictions = []
            x1 = x1.reshape((x1.shape[0],x1.shape[1],1))

            predict = model_regional.predict([seq_static, x1,x2]) 
            predictions.append(predict)
            dates.append(date) 
            ed_peak.append(np.max(np.array(predictions).flatten()))
            ed_nse.append(r2_score(np.array(y).flatten(), np.array(predictions).flatten()))

            # peak hour
            hour_pre = np.argmax(np.array(predictions).flatten())
            hour_obs = np.argmax(np.array(y).flatten())
            ed_hour.append((hour_pre - hour_obs))
            
            # peak discharge
            max_pre = np.max(np.array(predictions).flatten())
            max_obs = np.max(np.array(y).flatten())
            ed_volume.append((max_pre - max_obs)/max_obs)

        temp = np.array([dates, ed_peak, ed_hour, ed_volume]) # statistics for event-wise prediction at each catchment
        output = pd.DataFrame([temp[0,:],temp[1,:],temp[2,:],temp[3,:]], index=['date', 'flow', 'hour','ratio'])
        output.to_csv('/content/drive/My Drive/master_thesis/_code/regional_model/peak_pub/%s_peak_pub.csv' % (catchment))
        nse_table[catchment] = np.median(ed_nse) # collect NSE score for each basin in one test fold

In [ ]:
nse_table.to_csv( '/content/drive/My Drive/master_thesis/_code/regional_model/regional_data/regional_nse.csv')